In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
spark = SparkSession.builder.appName('DecisionTreeAndRandomForest').getOrCreate()
from pyspark.ml.feature import (VectorAssembler,OneHotEncoder,StringIndexer)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier

df = spark.read.csv('ksprojectsDataPreFinish.csv', header=True, inferSchema=True)
df.printSchema()
print(df.columns)

root
 |-- KsID : integer (nullable = true)
 |-- main_category: integer (nullable = true)
 |-- deadline: timestamp (nullable = true)
 |-- launched: timestamp (nullable = true)
 |-- state: integer (nullable = true)
 |-- usd_pledged_real: double (nullable = true)
 |-- usd_goal_real: double (nullable = true)
 |-- launchedYear: integer (nullable = true)
 |-- launchedMonth: integer (nullable = true)
 |-- launchedWeekday: integer (nullable = true)
 |-- campaignDur: integer (nullable = true)

['KsID ', 'main_category', 'deadline', 'launched', 'state', 'usd_pledged_real', 'usd_goal_real', 'launchedYear', 'launchedMonth', 'launchedWeekday', 'campaignDur']


In [2]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns)

,KsID,main_category,deadline,launched,state,usd_pledged_real,usd_goal_real,launchedYear,launchedMonth,launchedWeekday,campaignDur
0,1000002330,3,2015-10-09,2015-08-11,0,0.0,1533.95,2015,8,1,59
1,1000003930,1,2017-11-01,2017-09-02,0,2421.0,30000.00,2017,9,5,60
2,1000004038,1,2013-02-26,2013-01-12,0,220.0,45000.00,2013,1,5,45
3,1000007540,2,2012-04-16,2012-03-17,0,1.0,5000.00,2012,3,5,30
4,1000011046,1,2015-08-29,2015-07-04,0,1283.0,19500.00,2015,7,5,56


In [3]:
col_now = ['main_category','usd_pledged_real','usd_goal_real','launchedYear','launchedMonth','launchedWeekday', 'campaignDur']

In [4]:
assembler = VectorAssembler(inputCols = col_now, outputCol="features")

In [5]:
from pyspark.ml import Pipeline

# Then go through our steps. It's essentially sequential to the above.
pipeline = Pipeline(stages=[assembler])

# Now that we've got a number of steps, let's apply it to the DataFrame.
pipeline_model = pipeline.fit(df)

# Incorporate results into a new DataFrame.
pipe_df = pipeline_model.transform(df)

# Remove all variables other than features and label. 
pipe_df = pipe_df.select('state', 'features')

In [6]:
# Split our data. Note that the new DataFrame is being used.
train_data, test_data = pipe_df.randomSplit([0.7,0.3])
print("Training Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 258756
Test Dataset Count: 111180


In [10]:
predictions = dtModel.transform(test_data)
predictions.show(10)

+-----+--------------------+-------------+-----------+----------+
|state|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|    0|[1.0,0.0,20.0,201...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,50.0,201...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,50.0,201...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,50.0,201...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,60.0,201...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,60.0,201...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,70.57,20...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,78.43,20...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,79.19,20...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
|    0|[1.0,0.0,84.15,20...| [5591.0,0.0]|  [1.0,0.0]|       0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 10 rows



# Building Decision Tree Model

In [11]:
dt_model = DecisionTreeClassifier(featuresCol='features',labelCol='state')

# Fit the model.
dt_model = dt_model.fit(train_data)

# And evaluate the model using the test data.
results = dt_model.transform(test_data)

In [12]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="state", predictionCol="prediction", metricName="accuracy")

In [13]:
accuracy = evaluator.evaluate(results)

In [14]:
print ("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.9755801403130059


# Building Ramdom Forest Model

In [15]:
rf = RandomForestClassifier(featuresCol='features',labelCol='state')
# Train model.
rf_model = rf.fit(train_data)

# Make predictions.
predictions = rf_model.transform(test_data)

In [16]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="state", predictionCol="prediction", metricName="accuracy")

In [17]:
accuracy = evaluator.evaluate(predictions)

In [18]:
print ("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.9708580679978414


In [19]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0291419
